# CSV

[Comma-Separated Values (CSV)](https://en.wikipedia.org/wiki/Comma-separated_values) 파일은 쉼표로 값을 구분하는 구분된 텍스트 파일입니다. 파일의 각 줄은 데이터 레코드입니다. 

각 레코드는 쉼표로 구분된 하나 이상의 필드로 구성됩니다.

## CSVLoader

- CSV 데이터를 문서당 한 행씩 로드합니다.

In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# CSV 로더 생성
loader = CSVLoader(file_path="./data/titanic.csv")

# 데이터 로드
docs = loader.load()

print(len(docs))
print(docs[0].metadata)

891
{'source': './data/titanic.csv', 'row': 0}


In [2]:
print(docs[1].page_content)

PassengerId: 2
Survived: 1
Pclass: 1
Name: Cumings, Mrs. John Bradley (Florence Briggs Thayer)
Sex: female
Age: 38
SibSp: 1
Parch: 0
Ticket: PC 17599
Fare: 71.2833
Cabin: C85
Embarked: C


### CSV 파싱 및 로딩 커스터마이징

[csv module](https://docs.python.org/3/library/csv.html) 문서를 참조하여 지원되는 **csv args**에 대한 자세한 정보를 확인하세요.

In [3]:
# 컬럼정보:
# PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked

# CSV 파일 경로
loader = CSVLoader(
    file_path="./data/titanic.csv",
    csv_args={
        "delimiter": ",",  # 구분자
        "quotechar": '"',  # 인용 부호 문자
        "fieldnames": [
            "Passenger ID",
            "Survival (1: Survived, 0: Died)",
            "Passenger Class",
            "Name",
            "Sex",
            "Age",
            "Number of Siblings/Spouses Aboard",
            "Number of Parents/Children Aboard",
            "Ticket Number",
            "Fare",
            "Cabin",
            "Port of Embarkation",
        ],  # 필드 이름
    },
)

# 데이터 로드
docs = loader.load()

# 데이터 출력
print(docs[1].page_content)

Passenger ID: 1
Survival (1: Survived, 0: Died): 0
Passenger Class: 3
Name: Braund, Mr. Owen Harris
Sex: male
Age: 22
Number of Siblings/Spouses Aboard: 1
Number of Parents/Children Aboard: 0
Ticket Number: A/5 21171
Fare: 7.25
Cabin: 
Port of Embarkation: S


문서 전체를 XML 문서 형식으로 처리하려는 경우
- 참고: 0번째 문서는 헤더 정보이기 때문에 스킵합니다.

In [4]:
row = docs[1].page_content.split("\n")
row_str = "<row>"
for element in row:
    splitted_element = element.split(":")
    value = splitted_element[-1]
    col = ":".join(splitted_element[:-1])
    row_str += f"<{col}>{value.strip()}</{col}>"
row_str += "</row>"
print(row_str)

<row><Passenger ID>1</Passenger ID><Survival (1: Survived, 0: Died)>0</Survival (1: Survived, 0: Died)><Passenger Class>3</Passenger Class><Name>Braund, Mr. Owen Harris</Name><Sex>male</Sex><Age>22</Age><Number of Siblings/Spouses Aboard>1</Number of Siblings/Spouses Aboard><Number of Parents/Children Aboard>0</Number of Parents/Children Aboard><Ticket Number>A/5 21171</Ticket Number><Fare>7.25</Fare><Cabin></Cabin><Port of Embarkation>S</Port of Embarkation></row>


In [5]:
for doc in docs[1:]:
    row = doc.page_content.split("\n")
    row_str = "<row>"
    for element in row:
        splitted_element = element.split(":")
        value = splitted_element[-1]
        col = ":".join(splitted_element[:-1])
        row_str += f"<{col}>{value.strip()}</{col}>"
    row_str += "</row>"
    print(row_str)

<row><Passenger ID>1</Passenger ID><Survival (1: Survived, 0: Died)>0</Survival (1: Survived, 0: Died)><Passenger Class>3</Passenger Class><Name>Braund, Mr. Owen Harris</Name><Sex>male</Sex><Age>22</Age><Number of Siblings/Spouses Aboard>1</Number of Siblings/Spouses Aboard><Number of Parents/Children Aboard>0</Number of Parents/Children Aboard><Ticket Number>A/5 21171</Ticket Number><Fare>7.25</Fare><Cabin></Cabin><Port of Embarkation>S</Port of Embarkation></row>
<row><Passenger ID>2</Passenger ID><Survival (1: Survived, 0: Died)>1</Survival (1: Survived, 0: Died)><Passenger Class>1</Passenger Class><Name>Cumings, Mrs. John Bradley (Florence Briggs Thayer)</Name><Sex>female</Sex><Age>38</Age><Number of Siblings/Spouses Aboard>1</Number of Siblings/Spouses Aboard><Number of Parents/Children Aboard>0</Number of Parents/Children Aboard><Ticket Number>PC 17599</Ticket Number><Fare>71.2833</Fare><Cabin>C85</Cabin><Port of Embarkation>C</Port of Embarkation></row>
<row><Passenger ID>3</Pas

`source_column` 인자를 사용하여 각 행에서 생성된 문서의 출처를 지정하세요. 그렇지 않으면 모든 문서의 출처로 `file_path`가 사용됩니다.

이는 CSV 파일에서 로드된 문서를 출처를 사용하여 질문에 답하는 체인에 사용할 때 유용합니다.

In [6]:
loader = CSVLoader(
    file_path="./data/titanic.csv", source_column="PassengerId"
)  # CSV 로더 설정, 파일 경로 및 소스 컬럼 지정

docs = loader.load()  # 데이터 로드

print(docs[1])  # 데이터 출력

page_content='PassengerId: 2
Survived: 1
Pclass: 1
Name: Cumings, Mrs. John Bradley (Florence Briggs Thayer)
Sex: female
Age: 38
SibSp: 1
Parch: 0
Ticket: PC 17599
Fare: 71.2833
Cabin: C85
Embarked: C' metadata={'source': '2', 'row': 1}


## UnstructuredCSVLoader 

`UnstructuredCSVLoader`를 사용하여 테이블을 로드할 수도 있습니다. `UnstructuredCSVLoader`를 사용하는 한 가지 장점은 `"elements"` 모드에서 사용할 경우, 메타데이터에서 테이블의 HTML 표현이 제공된다는 것입니다.

In [ ]:
from langchain_community.document_loaders.csv_loader import UnstructuredCSVLoader

# 비구조화 CSV 로더 인스턴스 생성
loader = UnstructuredCSVLoader(file_path="./data/titanic.csv", mode="elements")

# 문서 로드
docs = loader.load()

# 첫 번째 문서의 HTML 텍스트 메타데이터 출력
print(docs[0].metadata["text_as_html"][:1000])

## DataFrameLoader

- Pandas는 Python 프로그래밍 언어를 위한 오픈 소스 데이터 분석 및 조작 도구입니다. 이 라이브러리는 데이터 과학, 머신러닝, 그리고 다양한 분야의 데이터 작업에 널리 사용되고 있습니다.

In [ ]:
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv("./data/titanic.csv")

첫 5개 행을 조회합니다.

In [ ]:
# 데이터프레임의 처음 다섯 행 조회
df.head()

In [ ]:
from langchain_community.document_loaders import DataFrameLoader

# 데이터 프레임 로더 설정, 페이지 내용 컬럼 지정
loader = DataFrameLoader(df, page_content_column="Name")

# 문서 로드
docs = loader.load()

# 데이터 출력
print(docs[0].page_content)

# 메타데이터 출력
print(docs[0].metadata)

In [ ]:
# 큰 테이블에 대한 지연 로딩, 전체 테이블을 메모리에 로드하지 않음
for row in loader.lazy_load():
    print(row)
    break  # 첫 행만 출력